<a href="https://colab.research.google.com/github/MeAmarP/opencv_snipps/blob/master/Helpful_Python_snippets_for_Image_Processing_using_OpenCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import sys
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
print('OpenCV-Python Lib Version:', cv2.__version__)
print('Python Version:',sys.version)

OpenCV-Python Lib Version: 3.4.3
Python Version: 3.6.8 (default, Jan 14 2019, 11:02:34) 
[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]]


## Know about Image data and its representation.
+ Read the given image
+ Find out Image size, Image datatype Image.

### Types of Images:
+ Binary 
+ Grayscale 
+ True Color or RGB

Note: RGB pixel format is quit standard among MATLAB or other image processing liberaies, but OpenCV uses BGR pixel format.

In [9]:
MainImgBin = cv2.imread('/content/cell_bin.png')
MainImgBGR = cv2.imread('/content/fruitbowl_rgb.jpg')
MainImgGray = cv2.imread('/content/alg_gry.jpg')

print('DatatypeClass of Image:',type(MainImgBGR))
print('Shape/Size of Binary Img:', MainImgBin.shape)
print('Shape/Size of Gray Img:', MainImgGray.shape)
print('Shape/Size of RGB Img:', MainImgBGR.shape)

DatatypeClass of Image: <class 'numpy.ndarray'>
Shape/Size of Binary Img: (476, 638, 3)
Shape/Size of Gray Img: (512, 512, 3)
Shape/Size of RGB Img: (625, 960, 3)


## Types of Images
+ Binary 
+ Grayscale 
+ True Color or RGB